# Figure 3 Demo: Random-Forest Based Activity Classification

*Disclaimer:  This sample code and the reported results are represent the processed reported in the Pilot Study.  When running these processing with ML models incorporating stochasticity, the results shown in this demonstration notebook may differ slightly from those in the Pilot Study.*

In [1]:
import os
import re
import numpy as np
import pandas as pd
import mne
import shap
from boruta import BorutaPy
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold

from tqdm import tqdm

___

## Data Loading

In [2]:
# Create a mapping from string type task ID to integer task ID
task_label_map = {'Angry': 0, 'Chewing': 1, 'Eye': 2, 'Eye-Iso': 3, 'In-Iso': 4, 'Jaw': 5, 'L Gaze-L': 6, \
                  'L Gaze-R': 7, 'Out-Iso': 8, 'Sad': 9, 'Smile-Iso': 10, 'Surprise': 11, 'Swallowing': 12, \
                  'Talk': 13, 'Up Gaze': 14, 'Wrinkle-Iso': 15}

reverse_task_label_map = {value: key for key, value in task_label_map.items()}

subject_ids = ['subject0', 'subject1', 'subject2', 'subject3', 'subject4', \
               'subject5', 'subject6', 'subject7', 'subject8', 'subject9']

In [3]:
# Create a dictionary of features and labels.  Each dictionary maps a subject's
# ID to a set of activity features or activity labels, respectively.
features = {sid: [] for sid in subject_ids}
labels = {sid: [] for sid in subject_ids}

# Read feature data from csv files
feature_data_dir = '../data/feature_data'
for sid in tqdm(subject_ids):
    for time in ['Morning', 'Evening']:
        df = pd.read_csv(os.path.join(feature_data_dir, sid, '{}_{}_variables.csv'.format(sid, time)))
        df['Task Label'] = df['Task Label'].apply(lambda x: task_label_map[x])
        
        # Remove Event Duration feature, as this feature is only applicable to
        # tasks performed for specified durations.
        df.drop(['Event Duration (s)'], axis=1, inplace=True)
        
        y = np.array(df['Task Label'])
        
        df.drop(['Task Label'], axis=1, inplace=True)
        X = df.to_numpy()
        
        features[sid].append(X)
        labels[sid].append(y)
        
    features[sid] = np.vstack(features[sid])
    labels[sid] = np.hstack(labels[sid])

100%|██████████| 10/10 [00:00<00:00, 50.40it/s]


___

## Activity Classification

In [4]:
v1_feature_inds = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, \
                            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30])

v2_feature_inds = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, \
                            14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, \
                            26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, \
                            38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, \
                            50, 51, 52, 53, 54, 55, 56, 157, 158, 159, 160])

v3_feature_inds = np.arange(161)

In [5]:
X = np.vstack([features[sid] for sid in subject_ids])
y = np.hstack([labels[sid] for sid in subject_ids])

shuffled_inds = np.random.permutation(X.shape[0])
X = X[shuffled_inds]
y = y[shuffled_inds]

# Perform a stratified split of the pooled set of actions 
# (from all subjects) into training and test sets
test_size = 0.2
test_inds = []
train_inds = []
for class_id in list(task_label_map.values()):
    class_sample_idxs = np.where(y==class_id)[0]
    class_sample_count = len(class_sample_idxs)
    class_sample_idxs = np.random.permutation(class_sample_idxs)
    test_inds.append(class_sample_idxs[:int(test_size*class_sample_count)])
    train_inds.append(class_sample_idxs[int(test_size*class_sample_count):])
train_inds = np.hstack(train_inds)
test_inds = np.hstack(test_inds)

X_train_v1, X_test_v1 = X[train_inds][:,v1_feature_inds], X[test_inds][:,v1_feature_inds]
X_train_v2, X_test_v2 = X[train_inds][:,v2_feature_inds], X[test_inds][:,v2_feature_inds]
X_train_v3, X_test_v3 = X[train_inds][:,v3_feature_inds], X[test_inds][:,v3_feature_inds]
y_train, y_test = y[train_inds], y[test_inds]

In [6]:
print('Test Set Classification Report Summary:')

training_data_versions = [X_train_v1, X_train_v2, X_train_v3]
testing_data_versions = [X_test_v1, X_test_v2, X_test_v3]

for i, [X_train, X_test] in enumerate(zip(training_data_versions, testing_data_versions)):
    train_feature_means = np.mean(X_train, axis=0)
    train_feature_stds = np.std(X_train, axis=0)

    X_train = (X_train-train_feature_means)/train_feature_stds
    X_test = (X_test-train_feature_means)/train_feature_stds

    clf = RandomForestClassifier(n_estimators=500)
    clf.fit(X_train, y_train)
    y_hat = clf.predict(X_test)

    print('\nClassification Performance (feature set version {}):'.format(i))
    print(classification_report(y_test, y_hat))

Test Set Classification Report Summary:

Classification Performance (feature set version 0):
              precision    recall  f1-score   support

           0       0.44      0.27      0.33        15
           1       0.94      1.00      0.97        16
           2       0.80      1.00      0.89        16
           3       0.43      0.19      0.26        16
           4       0.36      0.62      0.45        16
           5       0.94      1.00      0.97        16
           6       0.36      0.29      0.32        14
           7       0.46      0.43      0.44        14
           8       0.64      0.44      0.52        16
           9       0.28      0.31      0.29        16
          10       0.58      0.94      0.71        16
          11       0.60      0.38      0.46        16
          12       1.00      0.88      0.93        16
          13       0.88      1.00      0.94        15
          14       0.67      0.80      0.73        15
          15       0.15      0.12      0.1

___
## Feature Selection With Boruta

In [7]:
from boruta import BorutaPy

X_train, X_test = X[train_inds], X[test_inds]
train_feature_means = np.mean(X_train, axis=0)
train_feature_stds = np.std(X_train, axis=0)
X_train = (X_train-train_feature_means)/train_feature_stds
X_test = (X_test-train_feature_means)/train_feature_stds

clf_reduced = RandomForestClassifier(n_estimators=500)
feat_selector = BorutaPy(estimator=clf_reduced, n_estimators='auto', max_iter=100)
feat_selector.fit(X_train, y_train)

supported_features = np.array(df.columns)[feat_selector.support_]
print('Supported Features ({} total):'.format(len(supported_features)))
print(supported_features)
      
weak_support_features = np.array(df.columns)[feat_selector.support_weak_]
print('\nWeakly Supported Features ({} total):'.format(len(weak_support_features)))
print(weak_support_features)

Supported Features (102 total):
['Perceptible Onset Time (s)'
 'EMG Mean Absolute Amplitude - Channel 1 (uV)'
 'EMG Mean Absolute Amplitude - Channel 2(uV)'
 'EMG Max Absolute Amplitude - Channel 1 (uV)'
 'EMG Max Absolute Amplitude - Channel 2 (uV)'
 'EMG Peak Frequency Contractions - Channel 1'
 'EMG Peak Frequency Contractions - Channel 2'
 'EMG 10-33Hz Bandpower - Channel 1 (dB)'
 'EMG 33-56Hz Bandpower - Channel 1 (dB)'
 'EMG 56-79Hz Bandpower - Channel 1 (dB)'
 'EMG 79-102Hz Bandpower - Channel 1 (dB)'
 'EMG 102-125Hz Bandpower - Channel 1 (dB)'
 'EMG 10-33Hz Bandpower - Channel 2 (dB)'
 'EMG 33-56Hz Bandpower - Channel 2 (dB)'
 'EMG 56-79Hz Bandpower - Channel 2 (dB)'
 'EMG 79-102Hz Bandpower - Channel 2 (dB)'
 'EMG 102-125Hz Bandpower - Channel 2 (dB)'
 'EMG Amplitude Skew - Channel 1' 'EMG Amplitude Skew - Channel 2'
 'EMG Amplitude Kurtosis - Channel 1' 'EMG Amplitude Kurtosis - Channel 2'
 'EMG Variance - Channel 1' 'EMG Variance - Channel 2'
 'EMG 25th Percentile Absolute A

In [8]:
boruta_feature_inds = np.where(feat_selector.support_)[0]

clf = RandomForestClassifier(n_estimators=500)
clf.fit(X_train[:,boruta_feature_inds], y_train)
y_hat = clf.predict(X_test[:,boruta_feature_inds])

print('\nClassification Performance (Boruta feature set)')
print(classification_report(y_test, y_hat))


Classification Performance (Boruta feature set)
              precision    recall  f1-score   support

           0       0.36      0.27      0.31        15
           1       0.94      1.00      0.97        16
           2       1.00      0.88      0.93        16
           3       0.55      0.38      0.44        16
           4       0.44      0.50      0.47        16
           5       0.84      1.00      0.91        16
           6       0.75      0.64      0.69        14
           7       0.71      0.71      0.71        14
           8       0.58      0.44      0.50        16
           9       0.38      0.50      0.43        16
          10       0.52      0.81      0.63        16
          11       0.75      0.38      0.50        16
          12       0.89      1.00      0.94        16
          13       1.00      0.93      0.97        15
          14       0.62      0.67      0.65        15
          15       0.42      0.50      0.46        16

    accuracy                   